In [1]:
import os
import tensorflow as tf

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

In [3]:
directory=r"C:/Users/PythonTeam/Desktop/BALAJI/BALAJI/DL Projects/Federated_Deep_Learning_for_Monkeypox_Disease_Detection_on_GAN-Augmented_Dataset/archive/Monkeypox Skin Image Dataset"
train_dir=os.path.join(directory,'Train')
validation_dir=os.path.join(directory,'Validation')

In [4]:
Image_height,Image_width=224,224
Batch_size=32
Epochs=20

In [5]:
train_datagenerator=ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

In [6]:
train_generator=train_datagenerator.flow_from_directory(
    directory,
    target_size=(Image_height,Image_width),
    batch_size=Batch_size,
    class_mode='categorical',
    subset='training')

Found 618 images belonging to 4 classes.


In [7]:
validation_generator=train_datagenerator.flow_from_directory(
    directory,
    target_size=(Image_height,Image_width),
    batch_size=Batch_size,
    class_mode='categorical',
    subset='validation')

Found 152 images belonging to 4 classes.


In [8]:
def create_model(model):
    model.trainable=True
    x=model.output
    x=GlobalAveragePooling2D()(x)
    x=Dense(1024,activation='relu')(x)
    x=Dropout(0.5)(x)
    output_layer=Dense(4,activation='softmax')(x)
    models=Model(inputs=model.input,outputs=output_layer)
    return models

In [9]:
mode={
    'ResNet50V2':ResNet50V2(weights='imagenet',include_top=False,input_shape=(Image_height,Image_width,3))}

In [10]:
results={}
for name,base_model in mode.items():
    print(f"Training {name}...")
    model=create_model(base_model)
    model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
    Max=model.fit(
        train_generator,
        epochs=Epochs,
        validation_data=validation_generator,
        verbose=1,
        callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=3,min_lr=1e-6)])
    loss, accuracy = model.evaluate(validation_generator)
    print(f"{name} - Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}")
    results[name] = accuracy

Training ResNet50V2...


C:\Users\PythonTeam\anaconda3\envs\python39\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 303s 12s/step - accuracy: 0.4675 - loss: 1.2745 - val_accuracy: 0.7039 - val_loss: 0.8108 - learning_rate: 1.0000e-04
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 207s 10s/step - accuracy: 0.8167 - loss: 0.5162 - val_accuracy: 0.8158 - val_loss: 0.5754 - learning_rate: 1.0000e-04
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 209s 10s/step - accuracy: 0.8821 - loss: 0.3097 - val_accuracy: 0.8158 - val_loss: 0.6196 - learning_rate: 1.0000e-04
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 211s 11s/step - accuracy: 0.9427 - loss: 0.2081 - val_accuracy: 0.7697 - val_loss: 0.8289 - learning_rate: 1.0000e-04
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 204s 10s/step - accuracy: 0.9468 - loss: 0.1410 - val_accuracy: 0.7368 - val_loss: 0.9138 - learning_rate: 1.0000e-04
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 199s 10s/step - accuracy: 0.9497 - loss: 0.1302 - val_accuracy: 0.8026 - val_loss: 0.8039 - learning_rate: 2.0000e-05
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 199s 10s/step - accuracy: 